In [1]:
import re
import sys; sys.path.append("../")

import numpy as np
import pandas as pd

from lib.columns import rearrange_complaint_columns
from lib.textract import parse_textract_datetime
from lib.explore import print_df
from lib.match import (
    ColumnsIndex, StringSimilarity, JaroWinklerSimilarity, DateSimilarity, ThresholdMatcher
)
from clean.baton_rouge_pd_cprr import clean_18
from clean.baton_rouge_csd_pprr import clean_17 as clean_csd_17, clean_19 as clean_csd_19

In [2]:
pd_cprr = clean_18()
pd_cprr

../clean/baton_rouge_pd_cprr.py:104: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "Officer Name"] = df["Officer Name"].str.strip()\
../clean/baton_rouge_pd_cprr.py:111: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "Complaint"] = df.loc[:, "Complaint"]\
../clean/baton_rouge_pd_cprr.py:119: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "Action"] = df["Action"]\
../clean/baton_rouge_pd_cprr.py:126: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "Disposition"] = df["Disposition"].str.replace(r" ?- ", " ")\
../clean/baton_rouge_pd_cprr.py:155: FutureWarning: The default value of regex will change from True to False in a future version.
  dep = df.officer_name.str.replace(
../clean/baton_rouge_pd_cprr.py:158: FutureWarning: The default value of regex will

,investigation_status,action,disposition,department_code,department_desc,last_name,first_name,middle_initial,middle_name,receive_year,receive_month,receive_day,occur_year,occur_month,occur_day,tracking_number,uid,charges,agency,data_production_year
0,administrative review,exonerated,exonerated,P1254,cib street crimes unit,barcelona,jesse,r,,2018,1,2,2017,12,30,2018-002,4990106e2df85caab5910fe45de338ef,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2017
1,administrative review,not sustained,not sustained,P10280,cib narcotics,acree,jason,k,,2018,1,9,2018,1,9,2018-004,ddbd2be4c2f4c79dfd5aaa4b4e49212d,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2018
2,office investigation,resigned,office investigation,P10476,op service comm center,rimes,melissa,j,,2018,1,9,2017,12,3,2018-003,0383a0f9c3222535e9f91e316d9fa9a5,2:4 abuse of sick leave - 66,Baton Rouge PD,2017
3,administrative review,not sustained,not sustained,P10578,cib cib,hamadeh,yuseff,g,,2018,1,10,2018,1,10,2018-005,9d1a70ded8a2660295b9a4ad9cb357a5,3:20 use of force - 53 hard empty hand,Baton Rouge PD,2018
4,administrative review,not sustained,not sustained,P10252,patrol 2nd district,pollar jr,charles,g,,2018,1,11,2018,1,11,2018-006,8012c7dc3c5f5de7f2eae3850c8e7c95,3:20 use of force - 53 hard empty hand,Baton Rouge PD,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,administrative review,letter of reprimand,sustained,P1245,patrol 2nd district,clark,kenneth,m,,2019,1,30,2018,3,27,2019-003,80cd4f3a680dcb9966b25e86298b97c9,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2018
137,administrative review,letter of reprimand,not sustained,P1245,patrol 2nd district,clark,kenneth,m,,2019,1,30,2018,3,27,2019-003,80cd4f3a680dcb9966b25e86298b97c9,3:18 insubordination - 43,Baton Rouge PD,2018
138,administrative review,letter of reprimand,sustained,P1245,patrol 2nd district,clark,kenneth,m,,2019,1,30,2018,3,27,2019-003,80cd4f3a680dcb9966b25e86298b97c9,2:12 conduct unbecoming an officer - 21,Baton Rouge PD,2018
139,administrative review,not sustained,not sustained,P10639,patrol 1st district,robertson,jason,r,,2018,1,26,2018,1,26,2018-010,adcf8ba6410ae8beb2311493fb0a8f29,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2018


In [9]:
pd_cprr[pd_cprr.duplicated(subset=['tracking_number', 'uid', 'charges'])]

,investigation_status,action,disposition,department_code,department_desc,last_name,first_name,middle_initial,middle_name,receive_year,receive_month,receive_day,occur_year,occur_month,occur_day,tracking_number,uid,charges,agency,data_production_year
51,administrative review,peer intervention training,sustained,P10528,patrol 1st district,tucker,candice,m,,2018,8,2,2018,7,23,2018-034,14b51269473f616670f95975bd406f42,3:20 use of force - 53 hard empty hand,Baton Rouge PD,2018
52,administrative review,peer intervention training,sustained,P10528,patrol 1st district,tucker,candice,m,,2018,8,2,2018,7,23,2018-034,14b51269473f616670f95975bd406f42,2:2 command of temper - 13,Baton Rouge PD,2018
53,administrative review,verbal judo training,not sustained,P10528,patrol 1st district,tucker,candice,m,,2018,8,2,2018,7,23,2018-034,14b51269473f616670f95975bd406f42,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2018
54,administrative review,verbal judo training,sustained,P10528,patrol 1st district,tucker,candice,m,,2018,8,2,2018,7,23,2018-034,14b51269473f616670f95975bd406f42,2:12 conduct unbecoming an officer - 21,Baton Rouge PD,2018
55,administrative review,verbal judo training,sustained,P10528,patrol 1st district,tucker,candice,m,,2018,8,2,2018,7,23,2018-034,14b51269473f616670f95975bd406f42,3:20 use of force - 53 hard empty hand,Baton Rouge PD,2018
56,administrative review,verbal judo training,sustained,P10528,patrol 1st district,tucker,candice,m,,2018,8,2,2018,7,23,2018-034,14b51269473f616670f95975bd406f42,2:2 command of temper - 13,Baton Rouge PD,2018
57,administrative review,1-day suspension,not sustained,P10524,patrol 1st district,poland,untorio,d,,2018,8,2,2018,7,23,2018-034,39a7a0ca13c7529bdad52ffbeb5a8ece,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2018
58,administrative review,1-day suspension,sustained,P10524,patrol 1st district,poland,untorio,d,,2018,8,2,2018,7,23,2018-034,39a7a0ca13c7529bdad52ffbeb5a8ece,2:14 dmvr violation - 68,Baton Rouge PD,2018
63,administrative review,21-day suspension,not sustained,P10528,patrol 1st district,tucker,candice,m,,2018,8,2,2018,7,23,2018-034,14b51269473f616670f95975bd406f42,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2018
64,administrative review,21-day suspension,sustained,P10528,patrol 1st district,tucker,candice,m,,2018,8,2,2018,7,23,2018-034,14b51269473f616670f95975bd406f42,2:12 conduct unbecoming an officer - 21,Baton Rouge PD,2018


In [4]:
pd_cprr[~(pd_cprr.action == 'office investigation')]

,investigation_status,action,disposition,department_code,department_desc,last_name,first_name,middle_initial,middle_name,receive_year,receive_month,receive_day,occur_year,occur_month,occur_day,tracking_number,uid,charges,agency,data_production_year
0,administrative review,exonerated,exonerated,P1254,cib street crimes unit,barcelona,jesse,r,,2018,1,2,2017,12,30,2018-002,4990106e2df85caab5910fe45de338ef,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2017
1,administrative review,not sustained,not sustained,P10280,cib narcotics,acree,jason,k,,2018,1,9,2018,1,9,2018-004,ddbd2be4c2f4c79dfd5aaa4b4e49212d,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2018
2,office investigation,resigned,office investigation,P10476,op service comm center,rimes,melissa,j,,2018,1,9,2017,12,3,2018-003,0383a0f9c3222535e9f91e316d9fa9a5,2:4 abuse of sick leave - 66,Baton Rouge PD,2017
3,administrative review,not sustained,not sustained,P10578,cib cib,hamadeh,yuseff,g,,2018,1,10,2018,1,10,2018-005,9d1a70ded8a2660295b9a4ad9cb357a5,3:20 use of force - 53 hard empty hand,Baton Rouge PD,2018
4,administrative review,not sustained,not sustained,P10252,patrol 2nd district,pollar jr,charles,g,,2018,1,11,2018,1,11,2018-006,8012c7dc3c5f5de7f2eae3850c8e7c95,3:20 use of force - 53 hard empty hand,Baton Rouge PD,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,administrative review,letter of reprimand,sustained,P1245,patrol 2nd district,clark,kenneth,m,,2019,1,30,2018,3,27,2019-003,80cd4f3a680dcb9966b25e86298b97c9,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2018
137,administrative review,letter of reprimand,not sustained,P1245,patrol 2nd district,clark,kenneth,m,,2019,1,30,2018,3,27,2019-003,80cd4f3a680dcb9966b25e86298b97c9,3:18 insubordination - 43,Baton Rouge PD,2018
138,administrative review,letter of reprimand,sustained,P1245,patrol 2nd district,clark,kenneth,m,,2019,1,30,2018,3,27,2019-003,80cd4f3a680dcb9966b25e86298b97c9,2:12 conduct unbecoming an officer - 21,Baton Rouge PD,2018
139,administrative review,not sustained,not sustained,P10639,patrol 1st district,robertson,jason,r,,2018,1,26,2018,1,26,2018-010,adcf8ba6410ae8beb2311493fb0a8f29,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2018


In [5]:
pd_cprr[pd_cprr.tracking_number == '2018-003']

,investigation_status,action,disposition,department_code,department_desc,last_name,first_name,middle_initial,middle_name,receive_year,receive_month,receive_day,occur_year,occur_month,occur_day,tracking_number,uid,charges,agency,data_production_year
2,office investigation,resigned,office investigation,P10476,op service comm center,rimes,melissa,j,,2018,1,9,2017,12,3,2018-003,0383a0f9c3222535e9f91e316d9fa9a5,2:4 abuse of sick leave - 66,Baton Rouge PD,2017
